In [1]:
import json, openai
import os, sys, re
from openai import OpenAI
from tqdm.notebook import tqdm
import time
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)
from config.keys import OPENAI_API_KEY

In [2]:
class Generator:
    def __init__(self):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.json_formatter = "Return the response in RFC8259 compliant JSON according to the ResponseFormat schema with no other text."
        self.message = [{
            "role": "system",
            "content": "You are a cyber-security programmer that can detect line numbers from the contract based on the instruction."
        }]
        self.output_formatter = """
This should instruct the model to output exactly the vulnerability lines, ensuring it doesn't output extra lines or large ranges that contain unrelated code.

Response Schema:
 [
    {
      "start_line": <exact_start_line_number>,
      "end_line": <exact_end_line_number>,
      "code": [
        "vulnerable line 1",
        "vulnerable line 2",
        "... (and so on)"
      ]
    }
 ]
 
** Do not use ```json or any other extra texts in the output. Include only the list of detected lines as the schema.
"""
        self.user_prefix = """You are given a record from a dataset containing smart contract vulnerability analyses. Each record contains an "id", "prompt", and "completion". The "prompt" includes both the instructions and the smart contract code snippet, while the "completion" provides the vulnerability analysis.

Your task is to extract the exact lines of code that are vulnerable from the smart contract code. If the contract is vulnerable, output the minimal range of lines (with exact start and end line numbers) and the code lines themselves in the JSON Response Schema format provided below. If the contract is safe, do not output any JSON (i.e. return an empty result).

Instructions:
- Use the provided numbered contract for vulnerability detection.
- If the completion analysis indicates that the contract is vulnerable (i.e. it does not state that the contract is safe), locate the vulnerable lines of code precisely.
- Do not include any extra commentary or unrelated code; only include the vulnerable lines.
"""
    def get_user_message(self, dataset_output, contract):
        self.user_content = f"""
{self.user_prefix}

This is the helping document to find the lines of vulnerable codes.
Guideline:
{dataset_output}

Smart contract code:
{contract}

Additional Note:
- Only output the minimal range of code lines that are directly vulnerable.
- Do not include any commentary or unrelated code.
- Follow the JSON Response Schema exactly:
---
{self.output_formatter}

###
"""
        self.user_message = {"role": "user", "content": self.user_content}
    def create_prompt(self, dataset_output, contract):
        self.get_user_message(dataset_output, contract)
        self.message.append(self.user_message)
        
    def generate(self):
        completion = self.client.chat.completions.create(
          model="gpt-4o-mini",
          messages = self.message,
          temperature=0.1,
          max_tokens=3200,
          top_p=1.,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        answer = completion.choices[0].message.content
        return answer, completion


In [3]:
dataset_name = "train_TrustLLM"
output_dir = os.path.join("..", "..", "data", "processed_data", dataset_name)
os.makedirs(output_dir, exist_ok=True)

contracts_dir = os.path.join(output_dir, "contracts")
os.makedirs(contracts_dir, exist_ok=True)

locs_dir = os.path.join(output_dir, "LOCs")
os.makedirs(locs_dir, exist_ok=True)

raw_dir = os.path.join("..", "..", "data", "dataset", "raw")
dataset_path = os.path.join(raw_dir, f"{dataset_name}.json")

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset file not found at: {dataset_path}")

In [13]:
with open(dataset_path, "r", encoding="utf-8") as file:
    data = json.load(file)
print("Total records:", len(data))
print(data[0]["prompt"])

Total records: 20885
Below is an instruction that describes a reasoning task.
Examine the underlying factors and suggest a reason given the label name.
### Instruction:
Please review the code and its calling relationships as the caller and the callee. Given the label name, please find out the reason.
### Input:
The function Vesting from the contract vest 
```Solidiy
function vest( address to, uint256 amount, uint256 vestPeriodInSeconds ) external returns (uint256 vestIdx) { require(vestPeriodInSeconds > 0, "Vesting: vestPeriodInSeconds == 0"); token.safeTransferFrom(msg.sender, address(this), amount); vestIdx = accountVestList[to].length; accountVestList[to].push( Vest({ amount: amount, vestPeriodInSeconds: vestPeriodInSeconds, creationTimestamp: now, withdrawnAmount: 0 }) ); }
```
### As a Caller:
Vesting calls these functions:
```
IERC20.safeTransferFrom
Vesting.address
```
### Response:
The input code is safe. Please state the reason. Let‘s think step by step.


In [5]:
ids = [record.get("id") for record in data]
if len(ids) == len(set(ids)):
    print("All record ids are unique.")
else:
    print("There are duplicate record ids.")

There are duplicate record ids.


In [12]:
end_contract = 1
vul_idx = 0  # Counter for vulnerable smart contracts

for idx, record in enumerate(tqdm(data[:end_contract], desc="Processing Records")):
    prompt_text = record.get("prompt", "")
    completion_text = record.get("completion", "")
    
    # Extract smart contract code from the prompt.
    # This regex now matches any code block enclosed in triple backticks regardless of the language tag.
    code_match = re.search(r"```[^\n]*\n(.*?)\n```", prompt_text, re.DOTALL)
    if code_match:
        contract_code = code_match.group(1).strip()
    else:
        print(f"Record {idx}: No smart contract code found in prompt.")
        continue
    ########## Add line numbers to the contract code
    
    lines = contract_code.split("\n")
    numbered_lines = [f"{i+1}: {line}" for i, line in enumerate(lines)]
    numbered_contract = "\n".join(numbered_lines)
    print(contract_code)
    # Save the contract with numbered lines
    sol_filepath = os.path.join(contracts_dir, f"{vul_idx}.sol")
    with open(sol_filepath, 'w', encoding='utf-8') as f:
        f.write(numbered_contract)

    print(f"Extracting vulnerability for Record {idx} (vulnerable contract index: {vul_idx})...")
    
    # Skip safe contracts
    if "appears to be safe" in completion_text.lower():
        print(f"Record {idx} is marked as safe. Skipping vulnerability extraction.")
        continue
    # Otherwise, if it does not mention "the issue", assume there is no clear vulnerability description.
    if "the issue" not in completion_text.lower():
        print(f"Record {idx} does not clearly indicate a vulnerability. Skipping vulnerability extraction.")
        continue
    
    # Save the vulnerable smart contract code to a .sol file using vul_idx for numbering
    sol_filepath = os.path.join(contracts_dir, f"{vul_idx}.sol")
    with open(sol_filepath, 'w', encoding='utf-8') as f:
        f.write(contract_code)
    
    # Add line numbers to the contract code for vulnerability mapping
    lines = contract_code.split("\n")
    numbered_lines = [f"{i+1}: {line}" for i, line in enumerate(lines)]
    numbered_contract = "\n".join(numbered_lines)
    
    print(f"Extracting vulnerability for Record {idx} (vulnerable contract index: {vul_idx})...")
    
    generator = Generator()
    # Use the 'completion' as the guideline for vulnerability extraction
    generator.create_prompt(completionL_text, numbered_contract)
    start_time = time.time()
    answer, _ = generator.generate()
    print(f"Vulnerability extraction for Record {idx} completed in {round(time.time() - start_time, 2)} seconds")
    
    # Parse the answer as JSON and save the vulnerability details if extraction was successful.
    try:
        vulnerability_data = json.loads(answer)
    except Exception as e:
        print(f"Error parsing vulnerability JSON for Record {idx}: {e}")
        continue
    
    json_filepath = os.path.join(locs_dir, f"{vul_idx}.json")
    with open(json_filepath, "w", encoding="utf-8") as json_file:
        json.dump(vulnerability_data, json_file, indent=4)
    print(f"Saved vulnerability data for Record {idx} to {json_filepath}")
    
    vul_idx += 1

print("Process completed!")

Processing Records:   0%|          | 0/1 [00:00<?, ?it/s]

function vest( address to, uint256 amount, uint256 vestPeriodInSeconds ) external returns (uint256 vestIdx) { require(vestPeriodInSeconds > 0, "Vesting: vestPeriodInSeconds == 0"); token.safeTransferFrom(msg.sender, address(this), amount); vestIdx = accountVestList[to].length; accountVestList[to].push( Vest({ amount: amount, vestPeriodInSeconds: vestPeriodInSeconds, creationTimestamp: now, withdrawnAmount: 0 }) ); }
Extracting vulnerability for Record 0 (vulnerable contract index: 0)...
Record 0 is marked as safe. Skipping vulnerability extraction.
Process completed!


In [7]:
print(generator.message[1]['content'])


You are given a record from a dataset containing smart contract vulnerability analyses. Each record contains an "id", "prompt", and "completion". The "prompt" includes both the instructions and the smart contract code snippet, while the "completion" provides the vulnerability analysis.

Your task is to extract the exact lines of code that are vulnerable from the smart contract code. If the contract is vulnerable, output the minimal range of lines (with exact start and end line numbers) and the code lines themselves in the JSON Response Schema format provided below. If the contract is safe, do not output any JSON (i.e. return an empty result).

Instructions:
- Use the provided numbered contract for vulnerability detection.
- If the completion analysis indicates that the contract is vulnerable (i.e. it does not state that the contract is safe), locate the vulnerable lines of code precisely.
- Do not include any extra commentary or unrelated code; only include the vulnerable lines.


Thi

In [8]:
print(answer)

[
    {
        "start_line": 5,
        "end_line": 5,
        "code": [
            "if ((error = ICERC20(token).mint(amount)) != NO_ERROR) {"
        ]
    }
]


In [9]:
text = "Below is an instruction that describes a reasoning task.\nCarefully assess the contributing factors and their interplay. Utilize the label name to form a coherent reasoning.\n### Instruction:\nPlease analyze the code function and its dependencies, including both incoming and outgoing calls. Considering the label name, identify the underlying cause.\n### Input:\nThe method FuseTokenAdapterV1 in the smart contract wrap\n```Solidiy\nfunction wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }\n```\n## Dependency Overview:\n### As a Caller:\nFunctions called by FuseTokenAdapterV1:\n```\nSafeERC20.safeTransferFrom\nFuseTokenAdapterV1.address\nSafeERC20.safeApprove\nFuseTokenAdapterV1.IERC20\nFuseTokenAdapterV1.ICERC20\nFuseTokenAdapterV1.FuseError\nSafeERC20.safeTransfer\nmodifier onlyAlchemist() { if (msg.sender != alchemist) { revert Unauthorized(\"Not alchemist\"); } _; }\n```\n### Response:\nGiven that the code is labeled vulnerable, let's determine the reason by breaking down the process."
# solidity_code = re.search(r'Solidiy(.*?)', text, re.DOTALL)

import re

# text = """
# ```solidity

# some texts

# ```solidity

# The rest
# """

# Regex pattern to match Solidity code blocks
pattern = r"```Solidiy\n(.*?)\n```"

matches = re.findall(pattern, text, re.DOTALL)

# Print extracted Solidity code blocks
for i, match in enumerate(matches, 1):
    print(match)


function wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }


In [10]:
print('''unction wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }\n```\n## Dependency Overview:\n### As a Caller:\nFunctions called by FuseTokenAdapterV1:\n```\nSafeERC20.safeTransferFrom\nFuseTokenAdapterV1.address\nSafeERC20.safeApprove\nFuseTokenAdapterV1.IERC20\nFuseTokenAdapterV1.ICERC20\nFuseTokenAdapterV1.FuseError\nSafeERC20.safeTransfer\nmodifier onlyAlchemist() { if (msg.sender != alchemist) { revert Unauthorized(\"Not alchemist\"); } _; }\n''')

unction wrap( uint256 amount, address recipient ) external onlyAlchemist returns (uint256) { SafeERC20.safeTransferFrom(underlyingToken, msg.sender, address(this), amount); SafeERC20.safeApprove(underlyingToken, token, amount); uint256 startingBalance = IERC20(token).balanceOf(address(this)); uint256 error; if ((error = ICERC20(token).mint(amount)) != NO_ERROR) { revert FuseError(error); } uint256 endingBalance = IERC20(token).balanceOf(address(this)); uint256 mintedAmount = endingBalance - startingBalance; SafeERC20.safeTransfer(token, recipient, mintedAmount); return mintedAmount; }
```
## Dependency Overview:
### As a Caller:
Functions called by FuseTokenAdapterV1:
```
SafeERC20.safeTransferFrom
FuseTokenAdapterV1.address
SafeERC20.safeApprove
FuseTokenAdapterV1.IERC20
FuseTokenAdapterV1.ICERC20
FuseTokenAdapterV1.FuseError
SafeERC20.safeTransfer
modifier onlyAlchemist() { if (msg.sender != alchemist) { revert Unauthorized("Not alchemist"); } _; }



In [ ]:
############# skip the above code!!!! and start from here

In [1]:
import json
import os
import re
from tqdm import tqdm

# Define dataset and directory paths
dataset_name = "train_TrustLLM"

# Paths for processed data and extracted contracts
output_dir = os.path.join("..", "..", "data", "processed_data", dataset_name)
os.makedirs(output_dir, exist_ok=True)

contracts_dir = os.path.join(output_dir, "contracts")
os.makedirs(contracts_dir, exist_ok=True)

locs_dir = os.path.join(output_dir, "LOCs")
os.makedirs(locs_dir, exist_ok=True)

# Path for raw dataset
raw_dir = os.path.join("..", "..", "data", "dataset", "raw")
dataset_path = os.path.join(raw_dir, f"{dataset_name}.json")

# Load dataset
with open(dataset_path, "r", encoding="utf-8") as file:
    data = json.load(file)

print("Total records:", len(data))
print("Example record:", data[0]["prompt"])

# Keywords to filter vulnerabilities
integer_keywords = ["integer overflow", "integer underflow", "addition overflow", "subtraction underflow", "arithmetic overflow"]
timestamp_keywords = ["timestamp", "block.timestamp", "now", "time manipulation"]

# Initialize lists for extracted vulnerabilities
integer_vulnerabilities = []
timestamp_vulnerabilities = []
loc_data = []  # List to store LOC information

# Vulnerable contract indexing
vul_idx = 0

# Process records
for idx, record in enumerate(tqdm(data, desc="Processing Records")):
    prompt_text = record.get("prompt", "")
    completion_text = record.get("completion", "")
    
    # Extract smart contract code using regex
    code_match = re.search(r"```(?:solidity|Solidiy)?\n(.*?)\n```", prompt_text, re.DOTALL)
    if code_match:
        contract_code = code_match.group(1).strip()
    else:
        print(f"Record {idx}: No Solidity code found.")
        continue
    
    # Number the contract code
    lines = contract_code.split("\n")
    numbered_lines = [f"{i+1}: {line}" for i, line in enumerate(lines)]
    numbered_contract = "\n".join(numbered_lines)

    # Skip safe contracts
    if "appears to be safe" in completion_text.lower():
        continue

    # Skip unclear vulnerability cases
    if "the issue" not in completion_text.lower():
        continue

    # Identify vulnerabilities
    is_integer_vuln = any(keyword in completion_text.lower() for keyword in integer_keywords)
    is_timestamp_vuln = any(keyword in completion_text.lower() for keyword in timestamp_keywords)

    # Save contract based on its vulnerability type
    if is_integer_vuln or is_timestamp_vuln:
        contract_filename = f"{vul_idx}.sol"
        contract_filepath = os.path.join(contracts_dir, contract_filename)

        # Save Solidity contract with numbered lines
        with open(contract_filepath, "w", encoding="utf-8") as f:
            f.write(numbered_contract)

        # Store LOC information
        loc_entry = {
            "id": record["id"],
            "contract_index": vul_idx,
            "lines_of_code": len(lines),
            "contract_file": contract_filepath
        }
        loc_data.append(loc_entry)

        # Create a record for the extracted vulnerabilities
        vuln_record = {
            "id": record["id"],
            "prompt": prompt_text,
            "completion": completion_text,
            "contract_file": contract_filepath
        }

        if is_integer_vuln:
            integer_vulnerabilities.append(vuln_record)

        if is_timestamp_vuln:
            timestamp_vulnerabilities.append(vuln_record)

        # Print extracted contract
        print(f"\nExtracted Contract {vul_idx} (Record {idx}):\n")
        print(numbered_contract)

        # Increment vulnerability index
        vul_idx += 1

# Save extracted vulnerabilities into separate JSON files
integer_json_path = os.path.join(output_dir, "integer_vulnerabilities.json")
timestamp_json_path = os.path.join(output_dir, "timestamp_vulnerabilities.json")
loc_json_path = os.path.join(locs_dir, "loc_data.json")

with open(integer_json_path, "w", encoding="utf-8") as f:
    json.dump(integer_vulnerabilities, f, indent=4)

with open(timestamp_json_path, "w", encoding="utf-8") as f:
    json.dump(timestamp_vulnerabilities, f, indent=4)

# Save LOC data as JSON
with open(loc_json_path, "w", encoding="utf-8") as f:
    json.dump(loc_data, f, indent=4)

print("\nExtraction complete!")
print(f"Saved {len(integer_vulnerabilities)} integer overflow/underflow cases in {integer_json_path}")
print(f"Saved {len(timestamp_vulnerabilities)} timestamp dependency cases in {timestamp_json_path}")
print(f"Saved {len(loc_data)} contracts' LOC information in {loc_json_path}")


Total records: 20885
Example record: Below is an instruction that describes a reasoning task.
Examine the underlying factors and suggest a reason given the label name.
### Instruction:
Please review the code and its calling relationships as the caller and the callee. Given the label name, please find out the reason.
### Input:
The function Vesting from the contract vest 
```Solidiy
function vest( address to, uint256 amount, uint256 vestPeriodInSeconds ) external returns (uint256 vestIdx) { require(vestPeriodInSeconds > 0, "Vesting: vestPeriodInSeconds == 0"); token.safeTransferFrom(msg.sender, address(this), amount); vestIdx = accountVestList[to].length; accountVestList[to].push( Vest({ amount: amount, vestPeriodInSeconds: vestPeriodInSeconds, creationTimestamp: now, withdrawnAmount: 0 }) ); }
```
### As a Caller:
Vesting calls these functions:
```
IERC20.safeTransferFrom
Vesting.address
```
### Response:
The input code is safe. Please state the reason. Let‘s think step by step.


Processing Records:   3%|███                                                                                                          | 579/20885 [00:00<00:03, 5774.11it/s]


Extracted Contract 0 (Record 65):

1: function setTimeLock(address _newAddress) external onlyOwner { emit SetTimeLock(TIME_LOCK_ADDRESS, _newAddress); TIME_LOCK_ADDRESS = _newAddress; }

Extracted Contract 1 (Record 66):

1: function setTimeLock(address _newAddress) external onlyOwner { emit SetTimeLock(TIME_LOCK_ADDRESS, _newAddress); TIME_LOCK_ADDRESS = _newAddress; }

Extracted Contract 2 (Record 67):

1: function setTimeLock(address _newAddress) external onlyOwner { emit SetTimeLock(TIME_LOCK_ADDRESS, _newAddress); TIME_LOCK_ADDRESS = _newAddress; }

Extracted Contract 3 (Record 68):

1: function setTimeLock(address _newAddress) external onlyOwner { emit SetTimeLock(TIME_LOCK_ADDRESS, _newAddress); TIME_LOCK_ADDRESS = _newAddress; }

Extracted Contract 4 (Record 69):

1: function setTimeLock(address _newAddress) external onlyOwner { emit SetTimeLock(TIME_LOCK_ADDRESS, _newAddress); TIME_LOCK_ADDRESS = _newAddress; }

Extracted Contract 5 (Record 75):

1: function add( uint256 _all

Processing Records:   8%|████████▊                                                                                                   | 1700/20885 [00:00<00:04, 4793.86it/s]


Extracted Contract 99 (Record 1004):

1: function createLot(uint256 _sdAmount) external override whenNotPaused { lots[nextLot].startBlock = block.number; lots[nextLot].endBlock = block.number + duration; lots[nextLot].sdAmount = _sdAmount; LotItem storage lotItem = lots[nextLot]; if (!IERC20(staderConfig.getStaderToken()).transferFrom(msg.sender, address(this), _sdAmount)) { revert SDTransferFailed(); } emit LotCreated(nextLot, lotItem.sdAmount, lotItem.startBlock, lotItem.endBlock, bidIncrement); nextLot++; }

Extracted Contract 100 (Record 1015):

1: function initiateWithdrawal(uint256 _amount) external override { require(balanceOf(msg.sender) >= _amount, "Insufficient balance"); _accountToWithdrawalRequest[msg.sender].amount = _amount; _accountToWithdrawalRequest[msg.sender].blockNumber = block.number; }

Extracted Contract 101 (Record 1016):

1: function initiateWithdrawal(uint256 _amount) external override { require(balanceOf(msg.sender) >= _amount, "Insufficient balance"); _acco

...


Extracted Contract 1233 (Record 18328):

1: function mint(uint256 shares, address to) public virtual returns (uint256 amount) { amount = previewMint(shares); asset.safeTransferFrom(msg.sender, address(this), amount); _mint(to, amount); emit Deposit(msg.sender, to, amount, shares); afterDeposit(amount, shares); }

Extracted Contract 1234 (Record 18329):

1: function mint(uint256 shares, address to) public virtual returns (uint256 amount) { amount = previewMint(shares); asset.safeTransferFrom(msg.sender, address(this), amount); _mint(to, amount); emit Deposit(msg.sender, to, amount, shares); afterDeposit(amount, shares); }

Extracted Contract 1235 (Record 18585):

1: function _mint(address beneficiary, uint256 amount) internal returns (bool) { totalAvailableToNow += ((block.timestamp - lastEvent) * currentTotalInflation); uint256 newTotalMintedToNow = totalMintedToNow + amount; require(newTotalMintedToNow <= totalAvailableToNow, "Mintable amount exceeded"); totalMintedToNow = newTotalMi

Processing Records:  95%|█████████████████████████████████████████████████████████████████████████████████████████████████████▊     | 19875/20885 [00:03<00:00, 6246.66it/s]


Extracted Contract 1266 (Record 18976):

1: function RedeemTo(uint256[] memory lock_index, address to) public nonReentrant { uint256 total_to_redeem = 0; uint256 total_to_burn = 0; for(uint256 i; i < lock_index.length; i++){ uint256 index = lock_index[i]; UserLockInfo memory lock_info = _userLocks[msg.sender][index]; require(block.timestamp >= lock_info.time, "dMute::Redeem: INSUFFICIENT_LOCK_TIME"); require(lock_info.amount >= 0 , "dMute::Redeem: INSUFFICIENT_AMOUNT"); require(lock_info.tokens_minted >= 0 , "dMute::Redeem: INSUFFICIENT_MINT_AMOUNT"); total_to_redeem = total_to_redeem.add(lock_info.amount); total_to_burn = total_to_burn.add(lock_info.tokens_minted); _userLocks[msg.sender][index] = UserLockInfo(0,0,0); } require(total_to_redeem > 0, "dMute::Lock: INSUFFICIENT_REDEEM_AMOUNT"); require(total_to_burn > 0, "dMute::Lock: INSUFFICIENT_BURN_AMOUNT"); for(uint256 i = _userLocks[msg.sender].length; i > 0; i--){ UserLockInfo memory lock_info = _userLocks[msg.sender][i - 1]; if(l

Processing Records: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 20885/20885 [00:04<00:00, 5117.34it/s]



Extracted Contract 1340 (Record 20525):

1: function cancel(NounsDAOStorageV3.StorageV3 storage ds, uint256 proposalId) external { NounsDAOStorageV3.ProposalState proposalState = stateInternal(ds, proposalId); if ( proposalState == NounsDAOStorageV3.ProposalState.Canceled || proposalState == NounsDAOStorageV3.ProposalState.Defeated || proposalState == NounsDAOStorageV3.ProposalState.Expired || proposalState == NounsDAOStorageV3.ProposalState.Executed || proposalState == NounsDAOStorageV3.ProposalState.Vetoed ) { revert CantCancelProposalAtFinalState(); } NounsDAOStorageV3.Proposal storage proposal = ds._proposals[proposalId]; address proposer = proposal.proposer; NounsTokenLike nouns = ds.nouns; uint256 votes = nouns.getPriorVotes(proposer, block.number - 1); bool msgSenderIsProposer = proposer == msg.sender; address[] memory signers = proposal.signers; for (uint256 i = 0; i < signers.length; ++i) { msgSenderIsProposer = msgSenderIsProposer || msg.sender == signers[i]; votes += nouns.